We will follow 4 steps to build this agentic system-
1. First we ensure the neccessary frameworks and libraries are installed.
2. Next, we will write functions to load the resumes.
3. Then we use CrewAI to define agents and their respective tasks.
4. And finally we construct the crew to bundle all the agents and their tasks in a flow and review the output.

# Step 0: Install and Import Relevant Libraries

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [20]:
#!pip install PyMuPDF
#!pip install python-docx
#!pip install crewai crewai-tools

# Step 1: Loading the Resume

In [48]:
import fitz  # PyMuPDF for PDF processing
import docx  # python-docx for DOCX processing

In [49]:
def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file using PyMuPDF."""
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_text_from_docx(file_path):
    """Extracts text from a DOCX file using python-docx."""
    doc = docx.Document(file_path)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return "\n".join(fullText)

def extract_text_from_resume(file_path):
    """Determines file type and extracts text."""
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    else:
        return "Unsupported file format."

In [50]:
res1 = extract_text_from_resume('/Users/apoorv/Desktop/AV/Code/GAI/YT_projects/Resumes/Bruce Wayne.pdf')
print(res1)

Bruce Wayne  
985-555-5555 - first.last@selu.edu  
EDUCATION  
Southeastern Louisiana University, Hammond, LA May 2021 Bachelor of Science, 
Biological Sciences GPA: 3.5 Concentration: Microbiology and Molecular Biology  
Related Coursework: Genetics, Molecular Biology and Biotechnology, Research Problems I & II, Cell 
Biology  
LABORATORY SKILLS  
● Electron and light microscopy  
● Separation of compounds utilizing thin layer chromatography (TLC)  
● Gel electrophoresis and ELISA technique  
● Detailed anatomical knowledge gained through dissections  
WORK EXPERIENCE  
CVS Pharmacy, Hammond, LA May 2021 - present Pharmacy Technician  
● Developed an advanced understanding of the pharmaceutical industry language ● 
Communicated effectively with customers, physician offices, and drug representatives about 
questions regarding prescriptions and drug interactions  
● Supported pharmacist by productively assembling, stocking, and distributing medications ● 
Complied with rules, regulation

In [51]:
res2 = extract_text_from_resume('/Users/apoorv/Desktop/AV/Code/GAI/YT_projects/Resumes/Anita Sanjok.docx')
print(res2)

Anita Sanjok                                           

Dwarka, 110078| 123-456-7890 | anitaname@delhiuni.edu | linkedin.com/in/anita

EDUCATION
University of Delhi, Delhi 								               May 2025
Master of Arts in Education 

Amity University, Jaipur									   May 2018
Bachelor of Arts in Sociology

RELEVANT PROFESSIONAL EXPERIENCE

University of Delhi, Vishwavidyalaya, Delhi	               		                           January  2024 – Present
Youth Development Specialist/Research Assistant
Facilitate activities for groups of 20-25 students ages 14-18 to develop self-efficacy and confidence in 2-3 north campus colleges 
Collaborate with program director and 5 specialists to gather quantitative and qualitative data, conduct literature reviews, and perform assessments to evaluate program effectiveness
Code qualitative data from student insights in SPSS and determine themes in student experiences and reported findings to program director
Plan activities, share insights, and discu

# Step 2: Preparing the Agents and Task

- Agent represents an AI or human assistant with a specific role and goal.
- Task defines an objective that an agent needs to accomplish.
- And finally, Crew is used to bundle multiple agents and their respective tasks together to work and achieve the set objective.


In [52]:
import os
from crewai import Agent, Task, Crew

In [53]:
with open('/Users/apoorv/Desktop/AV/Code/GAI/keys/openai.txt', 'r') as file:
    openai_key = file.read()
    
os.environ['OPENAI_API_KEY'] = openai_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

- The first agent will provide feedback on the resume.
- The second agent will improve the resume based on the feedback
- And, the third and final agent would suggest appropriate jobs based on the the improved resume and specified location

In [21]:
# Agent 1: Resume Strategist
resume_feedback = Agent(
    role="Professional Resume Advisor",
    goal="Give feedback on the resume to make it stand out in the job market.",
    verbose=True,
    backstory="With a strategic mind and an eye for detail, you excel at providing feedback on resumes to highlight the most relevant skills and experiences."
    )

In [22]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_feedback_task = Task(
    description=(
        """Give feedback on the resume to make it stand out for recruiters. 
        Review every section, inlcuding the summary, work experience, skills, and education. Suggest to add relevant sections if they are missing.  
        Also give an overall score to the resume out of 10.  This is the resume: {resume}"""
    ),
    expected_output="The overall score of the resume followed by the feedback in bullet points.",
    agent=resume_feedback
)

In [23]:
# Agent 2: Resume Strategist
resume_advisor = Agent(
    role="Professional Resume Writer",
    goal="Based on the feedback recieved from Resume Advisor, make changes to the resume to make it stand out in the job market.",
    verbose=True,
    backstory="With a strategic mind and an eye for detail, you excel at refining resumes based on the feedback to highlight the most relevant skills and experiences."
)

In [24]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_advisor_task = Task(
    description=(
        """Rewrite the resume based on the feedback to make it stand out for recruiters. You can adjust and enhance the resume but don't make up facts. 
        Review and update every section, including the summary, work experience, skills, and education to better reflect the candidates abilities. This is the resume: {resume}"""
    ),
    expected_output= "Resume in markdown format that effectively highlights the candidate's qualifications and experiences",
    # output_file="improved_resume.md",
    context=[resume_feedback_task],
    agent=resume_advisor
)

In [25]:
from crewai_tools import SerperDevTool

In [26]:
with open('/Users/apoorv/Desktop/AV/Code/GAI/keys/serper.txt', 'r') as file:
    serper_key = file.read()

os.environ["SERPER_API_KEY"] = serper_key

search_tool = SerperDevTool()

In [27]:
# Agent 3: Researcher
job_researcher = Agent(
    role = "Senior Recruitment Consultant",
    goal = "Find the 5 most relevant, recently posted jobs based on the improved resume recieved from resume advisor and the location preference",
    tools = [search_tool],
    verbose = True,
    backstory = """As a senior recruitment consultant your prowess in finding the most relevant jobs based on the resume and location preference is unmatched. 
    You can scan the resume efficiently, identify the most suitable job roles and search for the best suited recently posted open job positions at the preffered location."""
    )

In [28]:
research_task = Task(
    description = """Find the 5 most relevant recent job postings based on the resume recieved from resume advisor and location preference. This is the preferred location: {location} . 
    Use the tools to gather relevant content and shortlist the 5 most relevant, recent, job openings""",
    expected_output=(
        "A bullet point list of the 5 job openings, with the appropriate links and detailed description about each job, in markdown format" 
    ),
#    output_file="relevant_jobs.md",
    agent=job_researcher
)

# Step 3: Creating the Crew and Reviwing the Output

In [29]:
crew = Crew(
    agents=[resume_feedback, resume_advisor, job_researcher],
    tasks=[resume_feedback_task, resume_advisor_task, research_task],
    verbose=True
)

2025-02-06 18:21:59,752 - 8528514944 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [37]:
result = crew.kickoff(inputs={"resume": res2, "location": 'New Delhi'})

# Agent: Professional Resume Advisor
## Task: Give feedback on the resume to make it stand out for recruiters. 
        Review every section, inlcuding the summary, work experience, skills, and education. Suggest to add relevant sections if they are missing.  
        Also give an overall score to the resume out of 10.  This is the resume: Anita Sanjok                                           

Dwarka, 110078| 123-456-7890 | anitaname@delhiuni.edu | linkedin.com/in/anita

EDUCATION
University of Delhi, Delhi 								               May 2025
Master of Arts in Education 

Amity University, Jaipur									   May 2018
Bachelor of Arts in Sociology

RELEVANT PROFESSIONAL EXPERIENCE

University of Delhi, Vishwavidyalaya, Delhi	               		                           January  2024 – Present
Youth Development Specialist/Research Assistant
Facilitate activities for groups of 20-25 students ages 14-18 to develop self-efficacy and confidence in 2-3 north campus colleges 
Collaborate with prog

In [ ]:
from IPython.display import Markdown, display

In [38]:
markdown_content = resume_feedback_task.output.raw.strip("```markdown").strip("```").strip()
# Display the Markdown content
display(Markdown(markdown_content))

Overall Score: 7/10  

Feedback:  
- **Contact Information**: Ensure your LinkedIn URL is hyperlinked in digital copies for easy access. Consider adding a professional title or a personal brand statement under your name to clarify your specialization (e.g., "Youth Development Specialist").  

- **Summary Section**: There is currently no summary section. Adding a summary can provide a snapshot of your expertise, experiences, and career goals. Aim for 2-3 impactful sentences that communicate your key strengths and values related to education and youth development.  

- **Education**: Your education section is clear and well-structured. You might want to include any relevant coursework or projects that support your expertise in education and youth development.  

- **Professional Experience**: 
  - Your bullet points are generally strong, but consider starting each with powerful action verbs (e.g., "Facilitated," "Collaborated," "Managed") to make your contributions stand out more.  
  - In your most recent role, quantify your achievements where possible (e.g., "Facilitated activities that improved student confidence by X% as measured by...").  
  - For your role as Program Coordinator, clarify your specific contributions and the impact of your workshops (e.g., "Resulted in a X% increase in college applications from participants").  

- **Additional Work Experience**: This section demonstrates your relevant experience, but more focus on accomplishments rather than responsibilities can enhance impact. For example, specify the number of workshops held or the percentage increase in student participation resulting from your initiatives.  

- **Skills Section**: This section looks good, but consider categorizing skills into both hard and soft skills (e.g., Project Management, Data Analysis, Communication). Also, highlight any specific interpersonal skills that reflect your ability to work with youth, such as mentorship or motivational skills.  

- **Volunteer Experience**: If applicable, consider adding a section for volunteer work. Experience in volunteer roles can demonstrate initiative and commitment to community and youth development.  

- **Formatting and Design**: Ensure consistency in fonts, bullet points, and spacing throughout the document. A more visually appealing format can enhance readability. Consider using a professional template to create a more polished look.  

By implementing the above suggestions, you can enhance the effectiveness of your resume, making it more appealing to recruiters and showcasing your qualifications comprehensively.

In [39]:
markdown_content = resume_advisor_task.output.raw.strip("```markdown").strip("```").strip()
# Display the Markdown content
display(Markdown(markdown_content))

# Anita Sanjok
Dwarka, 110078 | 123-456-7890 | anitaname@delhiuni.edu | [LinkedIn](linkedin.com/in/anita)  
**Professional Title:** Youth Development Specialist  

---

## Summary
Dedicated Youth Development Specialist with a passion for fostering self-efficacy and confidence in adolescents. Proven ability to design and implement educational programs that connect youth with critical resources and opportunities to enhance personal growth and academic success. Excellent collaborator, communicator, and data analyst, focused on driving program effectiveness through insightful evaluation.

---

## Education
**University of Delhi, Delhi**  
Master of Arts in Education (Expected May 2025)  
*Relevant Coursework: Child Psychology, Educational Leadership, Curriculum Development*

**Amity University, Jaipur**  
Bachelor of Arts in Sociology (May 2018)  
*Projects: Conducted qualitative research on community engagement in education*

---

## Relevant Professional Experience

**University of Delhi, Vishwavidyalaya, Delhi**  
*Youth Development Specialist / Research Assistant*  
January 2024 – Present  
- **Facilitated** engaging activities for groups of 20-25 students (ages 14-18), resulting in increased self-efficacy and confidence levels as measured by pre-and post-activity surveys.  
- **Collaborated** with 5 specialists and the program director to gather and analyze quantitative and qualitative data, conducting literature reviews to enhance program assessment and effectiveness.  
- **Coded** qualitative data using SPSS, identifying key themes in student experiences and presenting actionable insights to the program director, enhancing program strategies.  
- **Planned** and delivered weekly meetings to share program insights, ensuring continuous improvement and adaptation based on challenges faced by students.  

**Amity University, Jaipur**  
*Program Coordinator*  
August 2021 – May 2024  
- **Engaged** with groups of 20-25 high school students on college and career preparation, contributing to a 30% increase in college application submissions among participants.  
- **Hosted** workshops for 40-50 parents, improving understanding of the college application process, which directly influenced increased familial support in student outcomes.  
- **Partnered** with school administrators to implement logistics for workshops, ensuring effective outreach and participation from target demographics.  
- **Collected** and analyzed feedback via Google Forms to continuously refine workshop content, achieving a 95% satisfaction rate among attendees.  

**Amity University, Jaipur**  
*Program Coordinator, Office of Research and Sponsored Programs*  
June 2018 – July 2021  
- **Collaborated** with program directors and faculty to successfully implement engaging events, attracting 25-50 students per session.  
- **Managed** a budget for student activities using Microsoft Excel, generating monthly reports to optimize funding allocation and resource management.  
- **Created** and interpreted student experience surveys through Qualtrics, delivering quarterly insights that led to strategic program adjustments.  
- **Designed** promotional graphics with Canva, achieving a 75% open rate on email campaigns for co-curricular activities.  

---

## Additional Work Experience

**Amity University, Jaipur**  
*Program Assistant, Department of Sociology*  
August 2016 – May 2018  
- **Liaised** with academic advisors to organize 10+ career workshops and employer panels, enhancing student exposure to various career paths within sociology.  
- **Developed** marketing materials through Canva, increasing social media engagement by 20% and driving event attendance.

---

## Skills
**Technical Skills:**   
- Proficient in Microsoft Office Suite (Word, Excel, PowerPoint), SPSS, Qualtrics, Canva  
- Data Analysis, Budget Management, Program Evaluation  

**Interpersonal Skills:**  
- Project Management, Group Facilitation, Mentorship, Effective Communication  
- Bilingual in Spanish and English  

---

## Volunteer Experience (Optional)
*Include any relevant volunteering roles here that demonstrate commitment to community and youth development*  
```
This enhanced resume effectively highlights Anita's qualifications and relevant experience, utilizing action-oriented language and quantifiable achievements to capture the attention of recruiters.

In [40]:
markdown_content = research_task.output.raw.strip("```markdown").strip("```").strip()
# Display the Markdown content
display(Markdown(markdown_content))


Here are 5 recently posted job openings for Youth Development Specialist roles in Bengaluru:

- **[Youth Development Specialist, Alpha](https://www.crossover.com/job-roles/executive-leadership/india/city/bengaluru/c-a0q2j000000gocJAAQ/alpha-youth-development-specialist)**
  - **Description:** This role involves leading youth development projects aimed at improving academic and life skills of adolescents. The position requires a strategic outlook in program design and execution with a focus on measurable outcomes.
  - **Salary:** $150,000/year

- **[Program Coordinator, Youth Program - Quest Alliance](https://in.indeed.com/q-youth-work-l-bengaluru,-karnataka-jobs.html)**
  - **Description:** Responsible for planning, implementing, and evaluating youth programs. Experience in stakeholder engagement and data analysis will be crucial to enhancing program effectiveness.
  
- **[Youth Work Coordinator - JCPRO ACADEMY](https://in.indeed.com/q-youth-work-l-bengaluru,-karnataka-jobs.html)**
  - **Description:** Seeking a coordinator to facilitate youth engagement through workshops and community programs. Strong communication skills and experience in educational settings are preferred.

- **[Learning and Development Specialist](https://in.linkedin.com/jobs/learning-specialist-jobs-bengaluru)**
  - **Description:** This position focuses on developing educational programs tailored for youth development. Ideal candidates should exhibit strong analytical skills and a passion for teaching and mentoring young adults.

- **[Youth Development Professional - Various Openings](https://in.indeed.com/q-youth-development-professional-l-bengaluru,-karnataka-jobs.html)**
  - **Description:** Multiple organizations are hiring for roles related to youth development. Responsibilities include designing programs, collaborating with schools, and evaluating program outcomes to meet the needs of youths in the community.

Make sure to carefully consider each role and apply to those that align best with your qualifications and career aspirations!

# Conclusion

- Build a fourth agent that customizes the resume, based on the job you select.
- Build another agent in the end to generate a cover letter customized for the job you choose to apply,
- Build an agent that will help you prepare for the job you select.